In [1]:
year = 2025; day = 10

from aocd import get_data, submit
import numpy as np
data = get_data(year=year, day=day)

data=data.split("\n")
data = [d.split() for d in data]

def convert_button(but):
    new_but = [0]*10
    for i in but:
        new_but[i] = 1
    return tuple(new_but)

def parse_line(line):
    target = line[0][1:-1]
    target = tuple([1 if c == "#" else 0 for c in target])
    joltages = tuple(map(int, line[-1][1:-1].split(",")))
    buts = line[1:-1]
    buts = [tuple(map(int, b[1:-1].split(","))) for b in buts]
    buts = [convert_button(but)[:len(target)] for but in buts]
    return target, buts, joltages
data = [parse_line(line) for line in data]

In [2]:
def add(a, b):
    return tuple((i+j) % 2 for i, j in zip(a, b))

In [ ]:
presses = []
for line in data:
    target, buttons, joltages = line

    seen_states = set()
    current_states = [[0]*len(target)]
    for iteration in range(1_000):
        new_to_visit = set()
        is_done = False
        for s in current_states:
            for buts in buttons:
                new_s = add(s, buts)
                if new_s == target:
                    is_done = True
                    presses.append(iteration+1)
                    break
                if new_s not in seen_states:
                    new_to_visit.add(new_s)
                    seen_states.add(new_s)
            if is_done: break
        if is_done:
            break
        current_states = list(new_to_visit)

    else:
        raise ValueError("max iterations exceeded")
ans = sum(presses)

In [4]:
submit(ans, part="a", year=year, day=day)

Part a already solved with same answer: 461


In [5]:
%pip install z3-solver


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from z3 import Int, Optimize, Sum, And, sat
presses = 0
for line in data:
    target, buttons, joltages = line

    # we want to solve Ax = b, where x must be nonnegative and integer
    # whish I figured out this .T sooner :)
    A = np.array(buttons).T
    b = joltages

    n = len(A[0]) # number of variables
    m = len(b)    # number of constraints

    X = [ Int('x%d' % i) for i in range(n) ]
    s = Optimize() 
    # nonnegativity constraint
    s.add(And([ X[i] >= 0 for i in range(n) ]))
    for i in range(m):
        # sum of button count * button action must equal joltage
        s.add( Sum([ A[i][j]*X[j] for j in range(n) ]) == b[i])

    # minimize total presses
    s.minimize(Sum(X))

    if s.check() == sat:
        sol = [s.model().evaluate(X[i]) for i in range(n)]
        total = sum([sol[i].as_long() for i in range(n)])
        presses += total
    else:
        raise ValueError(f"No solution found {s=}")

In [7]:
submit(presses, part="b", day=day, year=year)

Part b already solved with same answer: 16386
